# Фреймворк PyTorch для разработки искусственных нейронных сетей

---

https://qudata.com/ml/ru/NN_Base_Torch_Graph.html

* Соглашение о именовании в PyTorch гласит, что любая функция вида xxx возвращает новый тензор, т.е. является immutable функцией. В противоположность ей функция вида xxx_ изменяет изначальный тензор, т.е. является mutable функцией. Последние ещё носят название inplace функций. 
Почти для любой immutable функции в PyTorch существует её собрат. Однако бывает и так, что функция существует лишь в каком-то одном варианте. По понятным причинам, функции, изменяющие размер тензора всегда являются immutable.

* print(f"Поддерживается ли CUDA : {torch.cuda.is_available()}")<br>
print(f'Количество гпу девайсов: {torch.cuda.device_count()}')<br>
print(f"Характеристики видеокарты : {torch.cuda.get_device_properties(0)}")<br>
print(f"Удаляем всю незанятую память через torch.cuda.empty_cache()")<br>

* Но следует запомнить что .cuda() immutable функция. Т.е. она возвращает новый тензор, а не перезаписывает существующий.

* requires_grad заразителен. Это означает, что когда тензор создается с помощью других тензоров, для параметра requires_grad результирующего тензора будет установлено значение True, если хотя бы один из тензоров, используемых для создания, имеет для параметра requires_grad значение True.

* Можно использовать функцию-член is_leaf, чтобы определить, является ли переменная листовым тензором или нет. Листовые тензоры - это тензоры, которые мы подаем в нашу систему вычислений

* Когда мы делаем backward делаем расчет всех градиентов сети (считаем градиенты всех листовых узлов). Если их не посчитать, не сделать backward, то функция `.grad` ничего нам не выдаст!

1) requires_grad

Это атрибут класса Tensor. По умолчанию это False. Это удобно, когда вам нужно заморозить некоторые слои и запретить им обновлять параметры во время тренировки. Вы можете просто установить для параметра requires_grad значение False, и эти тензоры не будут участвовать в графе вычислений.

2) torch.no_grad()

Когда мы вычисляем градиенты, нам нужно кэшировать входные значения и промежуточные функции, поскольку они могут потребоваться для вычисления градиента позже.  Если нам нужно сохранить эти значения для вычисления градиента во время обратного прохода, это повлияет на объем памяти, занимаемой сетью.

С использованием torch.no_grad мы выполняем вывод нашей нейронной сети и мы не вычисляем градиенты и, следовательно, нам не нужно хранить эти значения. Фактически, во время вывода графа вычислений создавать не нужно, так как это приведет к бесполезному потреблению памяти.

* Так как листовой тензор x не пересоздаётся, у него необходимо обнулять градиенты иначе они начнут суммироваться на следующих итерациях цикла (с каждой эпохой). Поэтому мы делаем `optimizer.zero_grad()`

---

- Материалы и ДЗ берите с ноутбуков Web*.ipynb
- 10 занятий, 2 дня в неделю
- Тайминг занятий ~1.5 часа

<!-- <img src='https://drive.google.com/uc?export=view&id=1v51-gWkPgQmtIhcGpmwuw81TGwMz7aM5'> -->
<img src='images/Pytorch_logo.png'>

### План курса

1. Введение в PyTorch. Тензоры, автодифференцирование
2. Feed-forward нейронные сети на Pytorch
3. Dataloader, Dataset в Pytorch. Продвинутые методы оптимизации
4. Сверточные сети в Pytorch. Классификация изображений. Предобученные сети в Pytorch
5. Составная лосс-функция. Сегментация изображений.
6. Сверточные сети применительно к текстовым задачам. Эмбеддинг-слои. Классификация новостей одномерными свертками.
7. Рекурентные нейронные сети. GRU, LSTM на Pytorch. Задача NER.
8. GAN на Pytorch.
9. Bert и Transformer на Pytorch
10. Face Detection and Emotion Recognition

# PyTorch, вводное занятие

### План занятия:

* Установка
* Тензоры
* Введение в синтаксис pytorch и Тензорные вычисления
* Вычислительный граф и Автоматическое диференцирование
* Погружаемся в детали
* Tensorflow vs PyTorch
* Где полученные знания можно применить

# 0. Установка

In [ ]:
#!pip3 install torch torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 1. Тензоры

Тензоры схожи с ndarrays в NumPy, с добавлением того, что тензоры могут быть использованы на GPU для ускорения вычислений.

### Тензоры

Тензор - основная структура данных в библиотеках машинного обучения, которая похожа на массив Numpy. Что-то вроде n-мерной матрицы или массива массивов.Тензоры обеспечивают ускорение различных математических операций. Эти операции при выполнении в большом количестве в глубоком обучении имеют огромное значение в скорости.

<!-- <img src='https://drive.google.com/uc?export=view&id=1pDiSoBIL8IBpIFq3R4OKGRW3YseBRvFm'> -->
<img src='images/scalar-vector-matrix-tensor.jpg'>


Визуализизация тензора с более чем двумя осями:

<!-- <img src='https://drive.google.com/uc?Export=view&id=1C6pu0iDx1Ugz2OMyE9d6KZH9IcN3-STG'> -->
<img src='images/3-axis_numpy.png'>
<!-- <img src='https://drive.google.com/uc?export=view&id=1cmVLwGNLc8fkDpmNZreTecMNCXgD6zGl'> -->
<img src='images/3-axis_front.png'>
<!-- <img src='https://drive.google.com/uc?export=view&id=1XiGSZVsVQrlH279eu2IK1bekSvccX6aT'> -->
<img src='images/3-axis_block.png'>

# 2. Введение в синтаксис pytorch и Тензорные вычисления

[База от pytorch](https://pytorch.org/tutorials/beginner/basics/intro.html)

In [1]:
from IPython import display
import numpy as np
import random
import torch

### 2.1 Тензоры в pytorch

Тип данных, хранимых тензором, отражается в имени его конструктора. Конструктор без параметров вернёт специальное значение — тензор без размерности, который нельзя использовать ни в каких операциях.

In [3]:
torch.FloatTensor()
a = torch.Tensor()
a

tensor([])

Типы тензоров в pytorch:

torch.HalfTensor      # 16 бит, с плавающей точкой  
torch.FloatTensor     # 32 бита,  с плавающей точкой  
torch.DoubleTensor    # 64 бита, с плавающей точкой  

torch.ShortTensor     # 16 бит, целочисленный, знаковый  
torch.IntTensor       # 32 бита, целочисленный, знаковый  
torch.LongTensor      # 64 бита, целочисленный, знаковый  

torch.CharTensor      # 8 бит, целочисленный, знаковый  
torch.ByteTensor      # 8 бит, целочисленный, беззнаковый  

torch.Tensor является сокращённым названием для torch.FloatTensor. Так же в последних версиях существует автоматическое приведение типов, если типы не сопоставимы:

In [4]:
a = torch.FloatTensor([1.0])
b = torch.DoubleTensor([2.0])
print(a)
print(b)
a * b

tensor([1.])
tensor([2.], dtype=torch.float64)


tensor([2.], dtype=torch.float64)

Но где-то могут возникать проблемы в виду разных типов. Для этого предусмотрена возможность явного приведения типов:

In [5]:
a = torch.IntTensor([1])
print(a.type())
a = a.byte()
print(a.type())

torch.IntTensor
torch.ByteTensor


In [6]:
a.float()

tensor([1.])

### 2.2 Немного о различиях в функциях

Соглашение о именовании в PyTorch гласит, что любая функция вида xxx возвращает новый тензор, т.е. является immutable функцией. В противоположность ей функция вида xxx_ изменяет изначальный тензор, т.е. является mutable функцией. Последние ещё носят название inplace функций. 
Почти для любой immutable функции в PyTorch существует её собрат. Однако бывает и так, что функция существует лишь в каком-то одном варианте. По понятным причинам, функции, изменяющие размер тензора всегда являются immutable.

По поводу всех функций прошу [сюда](https://pytorch.org/docs/master/tensors.html). А сейчас мы коснемся лишь самых важных.

### 2.3 Инициализация

Начнем с того, как мы можем задать наш тензор. Полный список функций можно посмотреть в [официальном источнике](https://pytorch.org/docs/stable/torch.html) под заголовком Creation Ops. Вот некоторые разные варианты:

In [7]:
a = [1. , 1.4 , 2.5]
a_tensor = torch.tensor(a)
print(f"type of tensor : ", a_tensor.type())
print(f"Simple way: {torch.tensor(a)}")
print(f"Via type : {torch.FloatTensor(a)}")
print(f"Zeros:\n {torch.zeros((2, 3))}")
print(f"Превращаем а в нули : {a_tensor.zero_()}")
print(f"Заполним тензор константой : {a_tensor.fill_(5)}")
print(f"Range: {torch.arange(0, 10)}")
print(f"Complicated range: {torch.arange(4, 12, 2)}")
print(f"Space: {torch.linspace(1, 4, 6)}")

type of tensor :  torch.FloatTensor
Simple way: tensor([1.0000, 1.4000, 2.5000])
Via type : tensor([1.0000, 1.4000, 2.5000])
Zeros:
 tensor([[0., 0., 0.],
        [0., 0., 0.]])
Превращаем а в нули : tensor([0., 0., 0.])
Заполним тензор константой : tensor([5., 5., 5.])
Range: tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
Complicated range: tensor([ 4,  6,  8, 10])
Space: tensor([1.0000, 1.6000, 2.2000, 2.8000, 3.4000, 4.0000])


### 2.4 Случайная выборка

Теперь про то, как мы можем генерировать случайнные значения в наших тензорах. Полный список функций можно посмотреть по ссылке выше под заголовком Random sampling.

In [7]:
print(f"From 0 to 1: {torch.rand(1)}")
print(f"Vector from 0 to 1: {torch.rand(5)}")
print(f"Vector from a normal distribution with mean 0 and variance 1: {torch.randn(2, 3)}")
print(f"Vector from 0 to 10: {torch.randint(10, size=(5,))}")
print(f"Непрерывное равномерное распределение : {a_tensor.uniform_(0, 5)}")

From 0 to 1: tensor([0.5226])
Vector from 0 to 1: tensor([0.8354, 0.7621, 0.1135, 0.0893, 0.7426])
Vector from a normal distribution with mean 0 and variance 1: tensor([[ 1.0185,  1.0558, -1.0793],
        [ 1.5435,  0.4924,  1.5574]])
Vector from 0 to 10: tensor([4, 6, 5, 1, 5])
Непрерывное равномерное распределение : tensor([0.8849, 1.0840, 2.4671])


### 2.5 Математические операции

С матричными операциями так же все аналогично с тем же numpy

In [25]:
a = torch.arange(10).type(torch.FloatTensor)
b = torch.linspace(-10, 10, 10)
print(f"a: {a}\nshape: {a.size()}")
print(f"b: {b}\nshape: {b.size()}")

a: tensor([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])
shape: torch.Size([10])
b: tensor([-10.0000,  -7.7778,  -5.5556,  -3.3333,  -1.1111,   1.1111,   3.3333,
          5.5556,   7.7778,  10.0000])
shape: torch.Size([10])


In [26]:
print(f"a + b: {a + b},\n a * b: {a * b}")
print(f"a + b: {a.add(b)},\n a * b: {a.mul(b)}") # вычитание sub, деление - div
print(f"a + b: {a.add_(b)},\n a * b: {a.mul_(b)}")

a + b: tensor([-10.0000,  -6.7778,  -3.5556,  -0.3333,   2.8889,   6.1111,   9.3333,
         12.5556,  15.7778,  19.0000]),
 a * b: tensor([ -0.0000,  -7.7778, -11.1111, -10.0000,  -4.4444,   5.5556,  20.0000,
         38.8889,  62.2222,  90.0000])
a + b: tensor([-10.0000,  -6.7778,  -3.5556,  -0.3333,   2.8889,   6.1111,   9.3333,
         12.5556,  15.7778,  19.0000]),
 a * b: tensor([ -0.0000,  -7.7778, -11.1111, -10.0000,  -4.4444,   5.5556,  20.0000,
         38.8889,  62.2222,  90.0000])
a + b: tensor([-10.0000,  -6.7778,  -3.5556,  -0.3333,   2.8889,   6.1111,   9.3333,
         12.5556,  15.7778,  19.0000]),
 a * b: tensor([100.0000,  52.7160,  19.7531,   1.1111,  -3.2099,   6.7901,  31.1111,
         69.7531, 122.7160, 190.0000])


In [27]:
a = torch.arange(10).type(torch.FloatTensor)

print(f"Экспонента : {a.exp()},\n {torch.exp(a)}, \n {a.exp_()}")
print(f"Логарифм : {a.log()}")
print(f"Модуль : {a.abs()}")

Экспонента : tensor([1.0000e+00, 2.7183e+00, 7.3891e+00, 2.0086e+01, 5.4598e+01, 1.4841e+02,
        4.0343e+02, 1.0966e+03, 2.9810e+03, 8.1031e+03]),
 tensor([1.0000e+00, 2.7183e+00, 7.3891e+00, 2.0086e+01, 5.4598e+01, 1.4841e+02,
        4.0343e+02, 1.0966e+03, 2.9810e+03, 8.1031e+03]), 
 tensor([1.0000e+00, 2.7183e+00, 7.3891e+00, 2.0086e+01, 5.4598e+01, 1.4841e+02,
        4.0343e+02, 1.0966e+03, 2.9810e+03, 8.1031e+03])
Логарифм : tensor([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])
Модуль : tensor([1.0000e+00, 2.7183e+00, 7.3891e+00, 2.0086e+01, 5.4598e+01, 1.4841e+02,
        4.0343e+02, 1.0966e+03, 2.9810e+03, 8.1031e+03])


In [28]:
print(f"Скалярное произведение: {a.dot(b)}")
print(f"Mean: {a.mean()}, STD: {a.std()}")
print(f"Sum: {a.sum()}, Min: {a.min()}, Max: {a.max()}")

Скалярное произведение: 111618.328125
Mean: 1281.830810546875, STD: 2571.337890625
Sum: 12818.30859375, Min: 1.0, Max: 8103.083984375


In [29]:
a.shape

torch.Size([10])

In [31]:
a

tensor([1.0000e+00, 2.7183e+00, 7.3891e+00, 2.0086e+01, 5.4598e+01, 1.4841e+02,
        4.0343e+02, 1.0966e+03, 2.9810e+03, 8.1031e+03])

In [32]:
print(f"Reshape:\n{a.reshape(-1, 1)}\nshape: {a.reshape(-1, 1).size()}")
c = a.reshape(-1, 1).repeat(1, 5)
print(f"Повторения:\n{c}\nshape: {c.size()}")
print(f"Транспонирование:\n{c.T}\nshape: {c.T.size()}")
print(f"Уникальные элементы: {torch.unique(c)}")

Reshape:
tensor([[1.0000e+00],
        [2.7183e+00],
        [7.3891e+00],
        [2.0086e+01],
        [5.4598e+01],
        [1.4841e+02],
        [4.0343e+02],
        [1.0966e+03],
        [2.9810e+03],
        [8.1031e+03]])
shape: torch.Size([10, 1])
Повторения:
tensor([[1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00],
        [2.7183e+00, 2.7183e+00, 2.7183e+00, 2.7183e+00, 2.7183e+00],
        [7.3891e+00, 7.3891e+00, 7.3891e+00, 7.3891e+00, 7.3891e+00],
        [2.0086e+01, 2.0086e+01, 2.0086e+01, 2.0086e+01, 2.0086e+01],
        [5.4598e+01, 5.4598e+01, 5.4598e+01, 5.4598e+01, 5.4598e+01],
        [1.4841e+02, 1.4841e+02, 1.4841e+02, 1.4841e+02, 1.4841e+02],
        [4.0343e+02, 4.0343e+02, 4.0343e+02, 4.0343e+02, 4.0343e+02],
        [1.0966e+03, 1.0966e+03, 1.0966e+03, 1.0966e+03, 1.0966e+03],
        [2.9810e+03, 2.9810e+03, 2.9810e+03, 2.9810e+03, 2.9810e+03],
        [8.1031e+03, 8.1031e+03, 8.1031e+03, 8.1031e+03, 8.1031e+03]])
shape: torch.Size([10, 5])
Тра

### 2.6 Индексирование

In [34]:
a = torch.arange(100).reshape(10, 10)
print(f"Array:\n{a}\nshape: {a.size()}")
print(f"Get first column: {a[:, 0]}")
print(f"Get last row: {a[-1, :]}")
print(f"Add new aхis:\n{a[:, np.newaxis]}\nshape: {a[:, np.newaxis].size()}")
print(f"Specific indexing:\n{a[4:6, 7:]}")

Array:
tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
        [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
        [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
        [40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
        [50, 51, 52, 53, 54, 55, 56, 57, 58, 59],
        [60, 61, 62, 63, 64, 65, 66, 67, 68, 69],
        [70, 71, 72, 73, 74, 75, 76, 77, 78, 79],
        [80, 81, 82, 83, 84, 85, 86, 87, 88, 89],
        [90, 91, 92, 93, 94, 95, 96, 97, 98, 99]])
shape: torch.Size([10, 10])
Get first column: tensor([ 0, 10, 20, 30, 40, 50, 60, 70, 80, 90])
Get last row: tensor([90, 91, 92, 93, 94, 95, 96, 97, 98, 99])
Add new aхis:
tensor([[[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9]],

        [[10, 11, 12, 13, 14, 15, 16, 17, 18, 19]],

        [[20, 21, 22, 23, 24, 25, 26, 27, 28, 29]],

        [[30, 31, 32, 33, 34, 35, 36, 37, 38, 39]],

        [[40, 41, 42, 43, 44, 45, 46, 47, 48, 49]],

        [[50, 51, 52, 53, 54, 55, 56, 57, 58, 59]],

  

In [38]:
a[1:4, 1:7]

tensor([[11, 12, 13, 14, 15, 16],
        [21, 22, 23, 24, 25, 26],
        [31, 32, 33, 34, 35, 36]])

### 2.7 Из numpy в pytorch и обратно

In [39]:
a = torch.normal(mean=torch.zeros(2,4))
a.numpy()

array([[ 1.382557  , -1.4951386 , -2.4048622 , -0.29943353],
       [ 0.16119671, -1.1046624 ,  0.13540387,  0.59006184]],
      dtype=float32)

In [40]:
b = np.random.normal(size=(2, 4))
torch.from_numpy(b)

tensor([[-0.3484,  1.7626, -0.6509, -0.1078],
        [ 1.9365,  0.5307,  1.0322, -0.8510]], dtype=torch.float64)

### 2.8 CUDA

torch.cuda - это пакет для поддержки CUDA. Он поддерживает такую же функциональность как и CPU, но использует CUDA ядра для вычислений. С полным функционалом можно ознакомиться [здесь](https://pytorch.org/docs/stable/cuda.html?highlight=cuda#module-torch.cuda)

In [54]:
print(f"Поддерживается ли CUDA : {torch.cuda.is_available()}")
print(f'Количество гпу девайсов: {torch.cuda.device_count()}')
print(f"Характеристики видеокарты : {torch.cuda.get_device_properties(0)}")
print(f"Удаляем всю незанятую память через torch.cuda.empty_cache()")

Поддерживается ли CUDA : True
Количество гпу девайсов: 1
Характеристики видеокарты : _CudaDeviceProperties(name='NVIDIA GeForce GTX 1050 Ti', major=6, minor=1, total_memory=4095MB, multi_processor_count=6)
Удаляем всю незанятую память через torch.cuda.empty_cache()


Давайте посмотрим на практике как работать с cuda. Допустим мы инициализуем два тензора:

In [55]:
a = torch.normal(mean=torch.zeros(2, 4))
b = torch.normal(mean=torch.zeros(2, 4))
print(f"a:\n{a}\nb:\n{b}")

a:
tensor([[-0.2601,  0.4187, -0.4258, -0.0193],
        [-2.6121,  0.7796, -0.7726,  0.9650]])
b:
tensor([[ 1.3236, -1.4302,  0.1473,  0.3842],
        [-0.3428,  0.4743,  1.5734,  0.9997]])


Наши тензоры автоматом загружены в память cpu. Но мы легко можем перевести их на cpu таким способом:

In [56]:
a = a.cuda()
a

tensor([[-0.2601,  0.4187, -0.4258, -0.0193],
        [-2.6121,  0.7796, -0.7726,  0.9650]], device='cuda:0')

Теперь, если мы попробуем сложить эти два тензора, то у нас вылезет ошибка, т.к. один тензор на cpu, а другой на cuda:

In [57]:
a + b

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

Мы не можем производить никакие операции с тензорами, находящимеся на разных устройствах. Что бы сложить их нам нужно оба тензора перевести на одно устройство:

In [58]:
a + b.cuda()

tensor([[ 1.0635, -1.0114, -0.2784,  0.3649],
        [-2.9549,  1.2539,  0.8009,  1.9647]], device='cuda:0')

In [59]:
a.cpu() + b

tensor([[ 1.0635, -1.0114, -0.2784,  0.3649],
        [-2.9549,  1.2539,  0.8009,  1.9647]])

In [60]:
(a + b.cuda()).cpu()

tensor([[ 1.0635, -1.0114, -0.2784,  0.3649],
        [-2.9549,  1.2539,  0.8009,  1.9647]])

Так же мы можем задать следующее определение устройства. Если  есть куда, то выбираем куду. В ином случае - цпу:

In [61]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


У каждого тензора есть поле device, которое по умолчанию стоит cpu. Но мы можем менять его при инициализации или в процессе использования:

In [62]:
torch.randn(10, 10, device=device)

tensor([[-0.7523,  0.1458,  0.6877, -0.6105,  0.3967, -0.4356, -0.0895, -1.7021,
         -0.2351, -0.5090],
        [ 0.6334,  0.9073, -0.1473,  0.1311,  1.9641,  0.1998,  0.8276, -0.3566,
         -0.0585,  1.2147],
        [-0.4250,  1.2266,  0.1751, -0.5352,  0.3035, -0.6051,  0.5058, -0.3088,
         -0.2941,  0.4484],
        [-0.5641,  0.8395,  0.4367, -2.1138,  1.3126,  0.2453, -1.1184, -2.0332,
          0.1577,  0.2995],
        [-0.5988, -0.1898,  0.2693, -1.1540, -0.5868,  1.2473, -0.5219,  1.3886,
          1.2898,  0.8061],
        [ 1.3003, -0.0402, -0.2402, -1.3817, -1.3264, -0.7834,  1.3596, -0.1069,
          0.3842, -0.3169],
        [-0.7577,  1.3303, -0.1360, -0.6954,  1.3184, -0.1138, -0.2531,  1.6230,
         -0.3967,  0.1974],
        [-0.8455,  0.1971,  0.6523, -1.3374,  0.0168,  0.2377, -0.5404,  2.0415,
          0.1871, -0.8823],
        [ 0.5259, -0.2481,  1.7387, -0.3607, -0.1660,  0.9403, -0.7076,  0.2578,
         -0.4730, -0.2271],
        [ 0.9086, -

In [64]:
a = torch.tensor((2 ,3))
print(a)

tensor([2, 3])


Переместить можно не только a.cuda(), но и так:

In [65]:
a.to(device)

tensor([2, 3], device='cuda:0')

Но следует запомнить что .cuda() immutable функция. Т.е. она возвращает новый тензор, а не перезаписывает существующий a:

In [68]:
a

tensor([2, 3])

In [69]:
a.device

device(type='cpu')

Как видим наш тензор a все на том же cpu. Что бы интерпретатор запомнил что a у нас на куде необходимо присвоить значение выражение в тензор:

In [70]:
a = a.cuda()

In [71]:
a

tensor([2, 3], device='cuda:0')

Проверяем, находится ли сейчас тензор на куде:

In [72]:
a.is_cuda

True

# 3. Вычислительный граф и Автоматическое диференцирование



### 3.1 Вычислительный граф

Вычислительный граф — это иллюстрированная запись какой-либо функции, состоящая из вершин и рёбер. Вершины (или узлы) — вычислительные операции, которые необходимо выполнить, а рёбра связывают их в определённую последовательность.

Автоматическое дифференцирование - строительный блок не только в Pytorch, но и в каждой другой DL библиотеке. Движок автоматического дифференцирования в Pytorch называет [Autograd](https://pytorch.org/docs/stable/autograd.html). 

Современные архитектуры нейронных сетей могут иметь миллионы обучающихся параметров. С вычислительной точки зрения тренировка сети состоит из двух фаз:

1) Прямой проход для вычисления значения функции потерь.  
2) Обратный проход для вычисления градиентов обучаемых параметров.

Прямой проход весьма прямолинеен: выход одного слоя является входом другого.
Обратный проход немного сложнее, поскольку он требует от нас использования цепного правила для вычисления градиентов весов относительно функции потерь.

Ниже представлен простой пример вычислительного графа для вычисления выражения $\sigma(x*w_1 + w_0)$. Можно разбить вычисление на следующие шаги:

<!-- <img src='https://drive.google.com/uc?export=view&id=1jCTO6zBGyE8sYkkSv_6NENdFOiwuJCMC' width=600> -->
<img src='images/computational_graph.png' width=600>

Преимущества использования вычислительного графа в том, что каждый узел является независимым функционирующим куском кода, если получит все необходимые входные данные. Это позволяет  оптимизировать производительность при выполнении расчетов, используя многоканальную обработку, параллельные вычисления.

### 3.2 PyTorch Autograd

Теперь, когда мы понимаем, что такое вычислительный граф, вернемся к PyTorch и разберемся, как это реализовано в PyTorch.

#### 3.2.1 Tензоры и requires_grad

Как мы видели выше, тензор - это структура данных, которая является фундаментальным строительным блоком PyTorch и они во многом похожи на массивы numpy, за исключением того, что в отличие от numpy, тензоры предназначены для использования преимуществ параллельных вычислений графического процессора(GPU).

In [73]:
import torch
tsr = torch.Tensor(3,5)
tsr

tensor([[8.4490e-39, 9.6428e-39, 1.1112e-38, 9.5511e-39, 1.0102e-38],
        [1.0286e-38, 1.0194e-38, 9.6429e-39, 9.2755e-39, 9.1837e-39],
        [9.3674e-39, 1.0745e-38, 1.0653e-38, 9.5510e-39, 1.0561e-38]])

Вот он Tensor похожий на numpy ndarray. Структура данных, которая позволяет быстро выполнять операции линейной алгебры. Что бы сделать тензор обучающимся и мы бы смогли вычислить его градиент, необходимо поставить его параметр requires_grad в значение True.

requires_grad можно менять как при инициализации тензора, так и после:

In [74]:
t1 = torch.randn((3,3), requires_grad=True)
print(t1.requires_grad)

True


In [75]:
t2 = torch.FloatTensor(3, 3)
print(t2.requires_grad)

False


In [76]:
t2.requires_grad = True
print(t2.requires_grad)

True


requires_grad заразителен. Это означает, что когда тензор создается с помощью других тензоров, для параметра requires_grad результирующего тензора будет установлено значение True, если хотя бы один из тензоров, используемых для создания, имеет для параметра requires_grad значение True.

*__Вопрос__: в каких ситуациях нам не нужен градиент для переменных?*


In [77]:
x = torch.ones(3, requires_grad=True)
x

tensor([1., 1., 1.], requires_grad=True)

В x у нас хранится информация о градиенте. Мы можем получить ее через метод grad:

In [78]:
x.grad

В данном случае ничего нет, т.к. мы никаких действий с нашим тензором не производили.

Создадим переменную на основе x:

In [79]:
z = (x**2) + 5.0*x # задание - посчитать производную руками

In [80]:
z

tensor([6., 6., 6.], grad_fn=<AddBackward0>)

Мы вызываем метод backward и передаем ей единичный тензор:

In [81]:
z.backward(x)

И тогда, когда мы выполнили эту функцию - мы вызываем градиент и получаем значение градиента в точке (1, 1, 1):

In [84]:
x.grad

tensor([7., 7., 7.])

<!-- <img src='https://drive.google.com/uc?export=view&id=1jCTO6zBGyE8sYkkSv_6NENdFOiwuJCMC' width=600> -->
<img src='images/computational_graph.png' width=600>

In [108]:
x = torch.FloatTensor([1])
w1 = torch.tensor([0.417], requires_grad=True)
w0 = torch.tensor([0.72], requires_grad=True)
a = x * w1
b = a + w0
sigma = torch.sigmoid(b)
mse = torch.nn.functional.mse_loss(sigma, x)
print(sigma, mse)
mse.backward()

tensor([0.7571], grad_fn=<SigmoidBackward>) tensor(0.0590, grad_fn=<MseLossBackward>)


In [110]:
w1.grad, w0.grad

(tensor([-0.0893]), tensor([-0.0893]))

In [111]:
x = torch.FloatTensor([1])
w1 = torch.tensor([0.417-0.0893], requires_grad=True)
w0 = torch.tensor([0.72-0.0893], requires_grad=True)
a = x * w1
b = a + w0
sigma = torch.sigmoid(b)
mse = torch.nn.functional.mse_loss(sigma, x)
print(sigma, mse)
mse.backward()

tensor([0.7228], grad_fn=<SigmoidBackward>) tensor(0.0768, grad_fn=<MseLossBackward>)


Ошибка уменьшилась

In [112]:
w1 = -10
w0 = 10
w1g = 0.0
w0g = 0.0

for i in range(30):
    x = torch.FloatTensor([1])
    w1 = torch.tensor([w1-w1g], requires_grad=True)
    w0 = torch.tensor([w0-w0g], requires_grad=True)
    a = x * w1
    b = a + w0
    sigma = torch.sigmoid(b)
    mse = torch.nn.functional.mse_loss(sigma, x)
    print(sigma, mse)
    mse.backward()
    w1g = w1.grad; w0g = w0.grad

tensor([0.5000], grad_fn=<SigmoidBackward>) tensor(0.2500, grad_fn=<MseLossBackward>)
tensor([0.6225], grad_fn=<SigmoidBackward>) tensor(0.1425, grad_fn=<MseLossBackward>)
tensor([0.7016], grad_fn=<SigmoidBackward>) tensor(0.0890, grad_fn=<MseLossBackward>)
tensor([0.7512], grad_fn=<SigmoidBackward>) tensor(0.0619, grad_fn=<MseLossBackward>)
tensor([0.7843], grad_fn=<SigmoidBackward>) tensor(0.0465, grad_fn=<MseLossBackward>)
tensor([0.8080], grad_fn=<SigmoidBackward>) tensor(0.0369, grad_fn=<MseLossBackward>)
tensor([0.8258], grad_fn=<SigmoidBackward>) tensor(0.0304, grad_fn=<MseLossBackward>)
tensor([0.8397], grad_fn=<SigmoidBackward>) tensor(0.0257, grad_fn=<MseLossBackward>)
tensor([0.8510], grad_fn=<SigmoidBackward>) tensor(0.0222, grad_fn=<MseLossBackward>)
tensor([0.8603], grad_fn=<SigmoidBackward>) tensor(0.0195, grad_fn=<MseLossBackward>)
tensor([0.8682], grad_fn=<SigmoidBackward>) tensor(0.0174, grad_fn=<MseLossBackward>)
tensor([0.8750], grad_fn=<SigmoidBackward>) tensor(0.0

Видим, что ошибка падает. И мы получили обученные веса:

In [113]:
w1, w0

(tensor([-8.7318], requires_grad=True), tensor([11.2682], requires_grad=True))

У каждого тензора есть атрибут grad_fn, который отсылается к функции (математическому оператору), создающему переменную. 

grad_fn будет равен None, если нет зависимых функций, к примеру, переменная `а`
ни от чего не зависит, нет ни какой функции, из которой бы получилась переменная а, а значит и grad_fn=None

In [114]:
a = torch.randn((3,3), requires_grad=True)

In [117]:
a.grad_fn

Или

In [118]:
a = torch.randn((3,3), requires_grad=True)

w1 = torch.randn((3,3), requires_grad=True)
w2 = torch.randn((3,3), requires_grad=True)
w3 = torch.randn((3,3), requires_grad=True)
w4 = torch.randn((3,3), requires_grad=True)

b = w1 * a
c = w2 * a

d = w3 * b + w4 * c

L = 10 - d

print("The grad fn for a is", a.grad_fn)
print("The grad fn for d is", d.grad_fn)

The grad fn for a is None
The grad fn for d is <AddBackward0 object at 0x000001DA530AF4F0>


Можно использовать функцию-член is_leaf, чтобы определить, является ли переменная листовым тензором или нет. Листовые тензоры - это тензоры, которые мы подаем в нашу систему вычислений. В нашем случае это а:

In [119]:
a.is_leaf, w1.is_leaf

(True, True)

In [39]:
L.is_leaf

False

#### 3.2.2 Function

Все математические операции в PyTorch реализуются классом [torch.nn.Autograd.Function](https://pytorch.org/docs/stable/autograd.html#function). У этого класса есть две важные функции-члены, на которые нам нужно обратить внимание.

Во-первых, это forward функция, которая просто вычисляет выходные данные, используя входные данные.

Функция backward принимает входящий градиент, исходящий от части сети перед ней. Как вы можете видеть, градиент, который должен распространяться в обратном направлении от функции f, - это, по сути, градиент, который передается обратно в f от слоев перед ним, умноженный на локальный градиент выходных данных f по отношению к его входам. Именно это и делает обратная функция.


Алгоритмически, вот как происходит обратное распространение с графом вычислений. (Не фактическая реализация, только пример):

```

def backward(incoming_gradients):
    self.Tensor.grad = incoming_gradients

    for inp in self.inputs:
        if inp.grad_fn is not None:
            new_incoming_gradients = incoming_gradient * local_grad(self.Tensor, inp)

            inp.grad_fn.backward(new_incoming_gradients)
        else:
            pass


```



Здесь self.Tensor - это, тензор, созданный Autograd.Function, который использовался в нашем примере.

Входящие градиенты и локальные градиенты были описаны выше.

----

Чтобы вычислить производные в нашей нейронной сети, мы обычно обращаемся к тензору, представляющему нашу потерю. Затем мы возвращаемся по графику, начиная с узла, представляющего grad_fn наших потерь.

Как описано выше, обратная функция рекурсивно вызывается по графику, когда мы возвращаемся. Однажды мы достигаем листового узла, поскольку grad_fn равен None, но прекращаем возвращение по этому пути.

Здесь следует отметить, что PyTorch выдает ошибку, если вы вызываете backward () для векторного тензора. Это означает, что вы можете выполнять обратный вызов только для тензорного скалярного значения. В нашем примере, если мы предположим, что a - тензор с векторным значением, и обратимся к L, он выдаст ошибку.

In [120]:
import torch 

a = torch.randn((3,3), requires_grad=True)

w1 = torch.randn((3,3), requires_grad=True)
w2 = torch.randn((3,3), requires_grad=True)
w3 = torch.randn((3,3), requires_grad=True)
w4 = torch.randn((3,3), requires_grad=True)

b = w1 * a 
c = w2 * a

d = w3 * b + w4 * c 

L = (10 - d)
print(L)

L.backward()

tensor([[10.0860, 10.0473, 10.5145],
        [15.1088, 10.1630, 10.2447],
        [ 9.5888, 10.3490,  9.4556]], grad_fn=<RsubBackward1>)


RuntimeError: grad can be implicitly created only for scalar outputs

Это потому, что градиенты могут быть вычислены относительно скалярных значений по определению. Вы не можете точно отличить вектор от другого вектора.

Если вы просто сделаете небольшое изменение в приведенном выше коде, установив L как сумму всех ошибок, наша проблема будет решена.

In [121]:
import torch 

a = torch.randn((3,3), requires_grad=True)

w1 = torch.randn((3,3), requires_grad=True)
w2 = torch.randn((3,3), requires_grad=True)
w3 = torch.randn((3,3), requires_grad=True)
w4 = torch.randn((3,3), requires_grad=True)

b = w1 * a 
c = w2 * a

d = w3 * b + w4 * c 

# Replace L = (10 - d) by 
L = (10 - d).sum()
print(L)

L.backward()

tensor(90.0507, grad_fn=<SumBackward0>)


Как только это будет сделано, вы можете получить доступ к градиентам, вызвав атрибут grad в Tensor.

In [123]:
w1.grad, w2.grad, w3.grad, w4.grad

(tensor([[ 0.1864,  0.0213, -0.3317],
         [-0.8386, -0.3114,  0.4508],
         [ 0.0332,  0.7609, -0.0391]]),
 tensor([[-0.1972,  0.0721,  0.0780],
         [-1.6017, -0.8483,  0.6889],
         [ 0.1592, -0.0867, -0.0236]]),
 tensor([[-0.1191,  0.0266, -0.2018],
         [ 0.7164,  0.3033,  0.9427],
         [-0.3986,  0.3447,  0.0041]]),
 tensor([[-1.1652,  0.0330, -0.3334],
         [ 1.0305,  0.0163,  0.9282],
         [ 0.0426,  0.0887,  0.0424]]))

#### 3.2.3 Некоторые хитрости

1) requires_grad

Это атрибут класса Tensor. По умолчанию это False. Это удобно, когда вам нужно заморозить некоторые слои и запретить им обновлять параметры во время тренировки. Вы можете просто установить для параметра requires_grad значение False, и эти тензоры не будут участвовать в графе вычислений.

2) torch.no_grad()

Когда мы вычисляем градиенты, нам нужно кэшировать входные значения и промежуточные функции, поскольку они могут потребоваться для вычисления градиента позже.  Если нам нужно сохранить эти значения для вычисления градиента во время обратного прохода, это повлияет на объем памяти, занимаемой сетью.

С использованием torch.no_grad мы выполняем вывод нашей нейронной сети и мы не вычисляем градиенты и, следовательно, нам не нужно хранить эти значения. Фактически, во время вывода графа вычислений создавать не нужно, так как это приведет к бесполезному потреблению памяти.

PyTorch предлагает для этой цели диспетчер контекста, называемый torch.no_grad.

In [ ]:
with torch.no_grad():
    # inference code goes here 
    pass

# 4. Погружаемся в детали

Все основные модули которые будут рассматриваться ниже находятся в [torch.nn](https://pytorch.org/docs/stable/nn.html#). Все кроме оптимизаторов - они находятся в [torch.optim](https://pytorch.org/docs/stable/optim.html)

### 4.1. Слои

#### Линейный слой (Линейное преобразование)

[pytorch doc](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)

$$y = xA^T + b$$

In [124]:
layer = torch.nn.Linear(
    in_features=3,
    out_features=2,
    bias=True
)

In [125]:
layer.weight, layer.bias

(Parameter containing:
 tensor([[-0.1985,  0.0117, -0.1543],
         [-0.4389,  0.3156,  0.5709]], requires_grad=True),
 Parameter containing:
 tensor([0.4102, 0.1979], requires_grad=True))

### 4.2 Алгоритм обучения в pytorch

In [126]:
import torch
from torch import nn
from torch import optim

1. Для начала нам нужна модель через которую мы будем прогонять данные и получать какой-то результат. Для этого возьмем линейное преобразование:

In [127]:
linear = nn.Linear(2, 2)

In [128]:
linear

Linear(in_features=2, out_features=2, bias=True)

У слоя в pytorch мы всегда можем посмотреть веса и отклонение:

In [129]:
print('w: ', linear.weight)
print('b: ', linear.bias)

w:  Parameter containing:
tensor([[ 0.2037, -0.6649],
        [ 0.3150,  0.6308]], requires_grad=True)
b:  Parameter containing:
tensor([0.3652, 0.0841], requires_grad=True)


2. Теперь нам нужно определить функцию ошибок для подсчета градиента:

In [131]:
criterion = nn.MSELoss()

3. Так же нам нужен оптимизатор который будет изменять веса нашей модели:

In [132]:
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

4. Нам нужны данные (х) и верные метки (y) по которым мы поймем правильные ли предсказания делает модель. (В данном случае у нас всего пара значений (данные, метки). О том как организовывать много данных ниже и в дальнейших вебинарах):

In [133]:
x = torch.randn(2, requires_grad=True)
y = torch.randn(2, requires_grad=False)

x, y

(tensor([-1.2367, -0.1541], requires_grad=True), tensor([-1.2242, -1.4554]))

5. Перед тем как считать градиенты и менять веса, нам нужно обнулить градиенты хранящиеся в свойстве тензора .grad. Для этого выполняем следующую строчку кода:

In [134]:
optimizer.zero_grad()

Зануление всех градиентов. После каждой эпохи нам нужно обнулить градиенты оптимизатора? Зачем? 

6. Затем делаем предсказание на наших данных х, получаем предсказание модели и сохраняем это предсказание в переменную pred:

In [135]:
pred = linear(x)

In [136]:
pred

tensor([ 0.2157, -0.4026], grad_fn=<AddBackward0>)

7. Переменная pred имеет ту же размерность, что и y. y - это наша правильная метка (ground truth). На этом этапе мы сравниваем предсказанное с реальным и получаем некую численную оценку этого через функцию потерь:

In [137]:
loss = criterion(pred, y)
print('loss: ', loss, ' \nloss_item :', loss.item())

loss:  tensor(1.5910, grad_fn=<MseLossBackward>)  
loss_item : 1.5909819602966309


Стоит отметить, что если мы посмотрим на переменную .grad наших весов и отклонения, то ничего не будет:

In [138]:
print('dL/dw: ', linear.weight.grad) 
print('dL/db: ', linear.bias.grad)

dL/dw:  None
dL/db:  None


Это потому что мы не начинали идти в обратном направлении и высчитывать градиенты.

8. Что ж, самое время это сделать. Проходим в обратном направлении и вычислим градиенты:

In [139]:
loss.backward()

Теперь если мы посмотрим на grad весов, то уже что-то увидим:

In [140]:
print('w: ', linear.weight.grad)
print('b: ', linear.bias.grad)

w:  tensor([[-1.7808, -0.2218],
        [-1.3020, -0.1622]])
b:  tensor([1.4400, 1.0528])


9. Теперь самое время поменять веса. Для этого надо сделать так называемый шаг оптимизатора. Здесь оптимизатор имея информацию о высчитанных градиентах и значениях весов меняет последние:

In [141]:
# Веса до
print('BEFORE:\n','w: ', linear.weight)
print('b: ', linear.bias, '\n')

# Делаем шаг оптимизатора
optimizer.step()

# Веса после
print('AFTER:\n''w: ', linear.weight)
print('b: ', linear.bias)

BEFORE:
 w:  Parameter containing:
tensor([[ 0.2037, -0.6649],
        [ 0.3150,  0.6308]], requires_grad=True)
b:  Parameter containing:
tensor([0.3652, 0.0841], requires_grad=True) 

AFTER:
w:  Parameter containing:
tensor([[ 0.2215, -0.6626],
        [ 0.3280,  0.6325]], requires_grad=True)
b:  Parameter containing:
tensor([0.3508, 0.0736], requires_grad=True)


#  5. Tensorflow vs PyTorch:

<!-- <img src='https://drive.google.com/uc?export=view&id=13SNw7d9JGT8lHMrKN2-mNgYrqKqkNXfG'> -->
<img src='images/12.png'>

* Определение графа - верно для старых версий, в Октябре 2019 года добавили поддержку подобного стиля программирования 

#### Различия:

PyTorch разработал Facebook Lab в 2016 году, здесь динамическое определение графа. А Tensorflow разрабатывался командой Google Brains в 2015 и до 2019 года граф определялся только статически.

В TensorFlow граф определяется статически перед запуском модели. Связь осуществляется с помощью объекта tf.Session и tf.Placeholder — тензорами, которые во время выполнения программы будут заменены внешними данными.

В PyTorch можно определять, изменять и выполнять узлы как вам угодно без дополнительных session- и placeholder-интерфейсов. Когда вы пишите на TensorFlow, иногда кажется, что с моделью можно связываться через несколько крошечных отверстий в кирпичной стене, за которой и прячется модель.

Существуют архитектуры нейронных сетей, которые получают преимущества от динамического подхода. RNN со статическим графом длина входной последовательности остается постоянной. Это означает, что необходимо зафиксировать длину предложения на максимальном значении, а все более мелкие последовательности заполнять нулями.

Но теперь и в Tensorflow и PyTorch есть возможность динамического определения графа.


<!-- <img src='https://drive.google.com/uc?export=view&id=1YceAfoUFkzUo_nHPwpwW8OuklFlQGMwu'> -->
<img src='images/debates.png'>

[Ссылка](https://chel-center.ru/python-yfc/2020/12/22/pytorch-protiv-tensorflow-dlya-vashego-proekta-glubokogo-obucheniya-python/) на статью где хорошо объясняется различие pytorch от tensorflow.

# 6. Где полученные знания можно применять

pytorch можно использовать в следующих областях:
* Распознавание и синтез аудио. Об этом [torchaudio](https://pytorch.org/audio/stable/index.html)  
* Так же с инструментом pytorch можно заниматься регрессионными проблемами, прогнозирования временных рядов. Например предсказывать цены на определенные валюты, акции, дома и т.д. [ссылка](https://www.machinelearningmastery.ru/lstm-for-time-series-prediction-de8aeb26f2ca/)
* [Задачи NLP](https://pytorchnlp.readthedocs.io/en/latest/index.html#)  
* Можно генерировать разную информацию от картинок до текста и аудио.

В общем pytorch можно применять везде, где используются нейронные сети.

[Бонус](https://www.8host.com/blog/kak-ustanovit-i-ispolzovat-pytorch/): зайдите по ссылке и долистайте до "Экосистема Pytorch"

# Домашнее задание


- Срок сдачи 7 дней
- Если не успеваете, то после истечения срока в 7 дней на странице ДЗ появляется кнопка о **самостоятельном продлении** сроков еще на 7 дней
- Если после 14 дней, то через техподдержку
- В выходные дни возможен ревьюер ДЗ


Ноутбук: https://colab.research.google.com/drive/1l_ngLK9Ltwkd8Gkv9WwQYGo_ctCs3HvY

# Дополнительные материалы
1. [Pytorch vs Tensorflow in 2020](https://towardsdatascience.com/pytorch-vs-tensorflow-in-2020-fe237862fae1)
2. [Официальная документация PyTorch](https://pytorch.org/tutorials/)

# Определения


**Вычислительный граф** — это иллюстрированная запись какой-либо функции, состоящая из вершин и рёбер. Вершины (или узлы) — вычислительные операции, которые необходимо выполнить, а рёбра связывают их в определённую последовательность.

Шаги обучения:
1. Проход по батчу
2. Обнуление градиента
3. Предсказание модели на батче
4. Подсчет ошибки
5. Подсчет градиентов
6. Шаг оптимизации
7. Логирование информации

---